### Analysing Used Car Data from Ebay

We are going to work on a dataset of used cars listed  from Ebay Kleinanzeigen, the German based version of Ebay.  

The original data was originally scraped from [Orges Leka database](https://www.kaggle.com/orgesleka/used-cars-database), The version of the dataset we will be working with is a sample of the first 50,000 data points to run the code faster and that will not affect our analysis.

In [1]:
import pandas as pd
import numpy as np
autos = pd.read_csv(r"C:\Users\moham\Documents\Documents\DataQuest\Ebay Project\autos.csv", encoding="Windows-1252")

In [2]:
autos.isnull().sum()

dateCrawled                0
name                       0
seller                     0
offerType                  0
price                      0
abtest                     0
vehicleType            37869
yearOfRegistration         0
gearbox                20209
powerPS                    0
model                  20484
kilometer                  0
monthOfRegistration        0
fuelType               33386
brand                      0
notRepairedDamage      72060
dateCreated                0
nrOfPictures               0
postalCode                 0
lastSeen                   0
dtype: int64

The dataset contains 20 columns, most of which are strings.
Some columns have null values, but none have more than ~20% null values.
The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

In [3]:
cols = ["date_crawled", "name", "seller", "offer_type", "dollar_price", "ab_test",
        "vehicle_type", "registration_year", "gearbox", "power_ps", "model",
       "kilometer", "registration_month", "fuel_type", "brand", "unrepaired_damage",
        "ad_created", "pictures_number", "postal_code",  "last_seen_online"]
autos.columns = cols

In [4]:
autos.head()

,date_crawled,name,seller,offer_type,dollar_price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,pictures_number,postal_code,last_seen_online
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
print(autos["pictures_number"].sum())
print(autos["ab_test"].unique())
print(autos["offer_type"].unique())
print(autos['seller'].unique())
autos.drop(["pictures_number", "ab_test", "offer_type", "seller"], axis=1, inplace=True)

0
['test' 'control']
['Angebot' 'Gesuch']
['privat' 'gewerblich']


Based on the information we got from the data within each column, we decide to delete the following ones:
 - pictures_number because all  values are equal to zero
 - ab_test takes two different values: 'test' and 'control' that are not relevant for our analysis
 - offer_type takes two different values 'Angebot' and 'Gesuch' that are not relevant for our analysis
 - seller takes only one value

In [6]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 16 columns):
date_crawled          371528 non-null object
name                  371528 non-null object
dollar_price          371528 non-null int64
vehicle_type          333659 non-null object
registration_year     371528 non-null int64
gearbox               351319 non-null object
power_ps              371528 non-null int64
model                 351044 non-null object
kilometer             371528 non-null int64
registration_month    371528 non-null int64
fuel_type             338142 non-null object
brand                 371528 non-null object
unrepaired_damage     299468 non-null object
ad_created            371528 non-null object
postal_code           371528 non-null int64
last_seen_online      371528 non-null object
dtypes: int64(6), object(10)
memory usage: 45.4+ MB


The df.infos() method gives us information about the number of observations, each column data type and the memory used by the df. We can have more details about numerical values with the df.describe() method.

In [7]:
autos.describe()

,dollar_price,registration_year,power_ps,kilometer,registration_month,postal_code
count,3.715280e+05,371528.000000,371528.000000,371528.000000,371528.000000,371528.00000
mean,1.729514e+04,2004.577997,115.549477,125618.688228,5.734445,50820.66764
std,3.587954e+06,92.866598,192.139578,40112.337051,3.712412,25799.08247
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,1067.00000
25%,1.150000e+03,1999.000000,70.000000,125000.000000,3.000000,30459.00000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,49610.00000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,71546.00000
max,2.147484e+09,9999.000000,20000.000000,150000.000000,12.000000,99998.00000


Based on min and max values of numerical data above, we notice that we can reduce the memory usage by storing the data in more optimal types. For example, prices, kilometers and registration years are all positive values, we can use unsigned integers to store them instead of int64 and reduce the memory allocation.

In [8]:
autos.dollar_price = autos.dollar_price.astype(np.uint32)
autos.kilometer = autos.kilometer.astype(np.uint32)
autos.power_ps = autos.power_ps.astype(np.uint16)
autos.registration_month = autos.registration_month.astype(np.uint8)
autos.registration_year = autos.registration_year.astype(np.uint16)
autos.postal_code = autos.postal_code.astype(np.uint32)

In [9]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 16 columns):
date_crawled          371528 non-null object
name                  371528 non-null object
dollar_price          371528 non-null uint32
vehicle_type          333659 non-null object
registration_year     371528 non-null uint16
gearbox               351319 non-null object
power_ps              371528 non-null uint16
model                 351044 non-null object
kilometer             371528 non-null uint32
registration_month    371528 non-null uint8
fuel_type             338142 non-null object
brand                 371528 non-null object
unrepaired_damage     299468 non-null object
ad_created            371528 non-null object
postal_code           371528 non-null uint32
last_seen_online      371528 non-null object
dtypes: object(10), uint16(2), uint32(3), uint8(1)
memory usage: 34.4+ MB


We then reduced the memory usage by 1.5 MB out of 6.1 MB, i.e. 25%, just by optimizing the types of numerical data for 50,000 rows. 

In [10]:
print(autos.dollar_price.value_counts().sort_index().head())
print(autos.dollar_price.unique().shape)
c_dollar_prices = autos.dollar_price.copy()
s_dollar_prices = c_dollar_prices.sort_values(ascending=False)
s_dollar_prices.index = autos.index
print(s_dollar_prices.head())

0    10778
1     1189
2       12
3        8
4        1
Name: dollar_price, dtype: int64
(5597,)
0    2147483647
1      99999999
2      99999999
3      99999999
4      99999999
Name: dollar_price, dtype: uint32


There is exactly 2381 different prices from $ 0 to $ 12345678. We can easily deduce that some of these values are wrong and will bias our analysis. Let's have a deeper check to decide which range of prices we will keep.

In [11]:
v_dollar_prices = autos[autos["dollar_price"] > 100000].copy()
v_dollar_prices.sort_values(by=['dollar_price'], ascending=False, inplace=True)
v_dollar_prices[['name', 'kilometer', 'dollar_price']].head(20)

,name,kilometer,dollar_price
129969,Kaufe_AUTOS_Zum_basteln,150000,2147483647
108987,Biete_Hier_zum_Tauschen_an_aber_nur_realistisc...,150000,99999999
160530,Golf_4_schrott,20000,99999999
56973,tausche_ford_mondeo_mk3_ghia_gegen_anderes_auto,150000,99999999
69747,Suche_Mercedes_Benz_r_klasse_cdi,150000,99999999
77520,Tausch_gegen_gleichwertiges,150000,99999999
325589,Tausche/_verkaufe_Golf_1_LX_mit_H_Zulassung,150000,99999999
127820,audi_a6_c5_avant,5000,99999999
280059,Kaufe_VW_Golf_1_auch_ohne_TÜV_________________...,150000,99999999
362782,Passat_35i_1.8_90ps_Kombi_frisch_lackiert_!,150000,99999999


### What happened this time?

The first part of the above operation selects the price column. When pandas selects a single column from a DataFrame, pandas creates a view and not a copy. A view just means that no new object has been created. df['price'] references the price column in the original DataFrame.

This is analogous to the list example where we assigned an entire list to a new variable. No new object is created, just a new reference to the one already in existence.

Since no new data has been created, the assignment will modify the original DataFrame.

### Why is a warning triggered when our operation completed successfully?

Pandas does not know if you want to modify the original DataFrame or just the first subset selection.

For instance, you could have selected the price column as a Series to do further analysis with it without affecting the original DataFrame.

To get an independent copy, we called the copy method. 

In [12]:
v_dollar_prices._is_copy is None

True

v_dollar_prices does not refer the original dataframe, we can then make new assignments without modifying the original dataframe and without getting the SettingWithCopy Warning.

In [13]:
autos.dollar_price.value_counts().sort_index().head(20)

0     10778
1      1189
2        12
3         8
4         1
5        26
7         3
8         9
9         8
10       84
11        5
12        8
13        7
14        5
15       27
16        2
17        5
18        3
19        3
20       51
Name: dollar_price, dtype: int64

There are a number of listings with prices below \$40, including about 1,500 at \$0. There are also a small number of listings with very high values, including 9 at around or over $1 million.

Given that eBay is an auction site, there could legitimately be items where the opening bid is \$1. The opening bid does not however represent the real value of a car, we then decide to fix the minimal price for a car to $500 because we need prices to be real if we want to make some comparisons between brands for example. We remove anything above \$245,000, since it seems that prices increase steadily to that number and then double to next number.

In [14]:
print((~autos["dollar_price"].between(500,245000)).sum() / autos.shape[0])
autos = autos[autos["dollar_price"].between(500,245000)]
v_dollar_prices = autos[autos["dollar_price"] >= 50000].copy()
v_dollar_prices.sort_values(by=['dollar_price'], ascending=False, inplace=True)
v_dollar_prices.head(20)

0.09745698843694149


,date_crawled,name,dollar_price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen_online
29866,2016-03-21 11:50:40,Porsche_997_GT2,245000,coupe,2008,manuell,530,911,10000,1,benzin,porsche,nein,2016-03-21 00:00:00,80999,2016-04-06 06:45:13
113181,2016-03-25 15:31:14,Ferrari_Testarossa_Sammlerzustand,239000,coupe,1990,manuell,390,NaN,20000,2,benzin,sonstige_autos,nein,2016-03-25 00:00:00,65232,2016-04-07 09:16:04
276665,2016-03-27 20:49:38,Lamborghini_Huracan_LP_610_4,238000,coupe,2015,automatik,610,NaN,5000,7,benzin,sonstige_autos,nein,2016-03-27 00:00:00,84371,2016-04-05 19:45:31
219951,2016-03-28 14:53:26,Mercedes_Benz_SLS_AMG_Roadster_AMG_SPEEDSHIFT_DCT,237500,cabrio,2013,automatik,571,andere,10000,6,benzin,mercedes_benz,nein,2016-03-28 00:00:00,33824,2016-04-02 19:17:58
55825,2016-03-29 20:47:24,Andere_Horch_830,235000,cabrio,1933,manuell,69,NaN,150000,7,NaN,sonstige_autos,nein,2016-03-29 00:00:00,74245,2016-03-31 14:44:47
270755,2016-03-21 18:57:32,Porsche_GT3RS,233000,coupe,2015,NaN,0,911,5000,10,benzin,porsche,nein,2016-03-21 00:00:00,76275,2016-03-21 19:41:21
226905,2016-03-31 08:57:50,Porsche_991_GT3_RS,230000,coupe,2016,automatik,0,911,5000,3,benzin,porsche,nein,2016-03-31 00:00:00,93444,2016-03-31 10:43:06
60482,2016-03-23 15:54:12,Porsche_991___Suche_991_R_bis_230TEUR,230000,coupe,2016,manuell,500,911,5000,3,NaN,porsche,nein,2016-03-23 00:00:00,80469,2016-03-23 15:54:12
291754,2016-03-26 13:46:04,Jaguar_Projekt_7_lim._Sondermodell,230000,cabrio,2016,NaN,575,andere,5000,3,benzin,jaguar,nein,2016-03-26 00:00:00,85276,2016-04-06 01:17:47
109280,2016-03-07 23:57:22,Tesla_Model_X_Founder_Series!!_P90D,229900,suv,2016,automatik,532,NaN,5000,3,elektro,sonstige_autos,nein,2016-03-07 00:00:00,34117,2016-03-17 21:45:54


~10% of the data have been removed which represents a important number of outliers.

In [15]:
autos.loc[(autos["vehicle_type"] == "kombi") & (autos["dollar_price"] > 50000), "dollar_price"]

1284       51000
4631       55999
7057       53900
15348      59000
15999      51593
17936      52999
36614      50699
43724      53900
46121      60000
47733      69990
54852      56800
63034      59799
66400      61000
74777      55990
78528      62900
83601      55555
85885      52500
92704      64990
98541     107000
98583      55000
103164     54900
108993    123456
123966     75990
131395    122990
133657     58900
138504     55900
148219     51900
157347     56500
163456     69000
168051     50500
           ...  
206388     61000
215174    145000
216550     64990
217159     54999
226965     56990
230653     60000
241625     64900
254124     58999
264521     65990
267894     56500
270901     53500
271558     51900
272238     69900
278772     63452
280848     64600
282941     59500
283240     58000
285159     51200
288185     56800
293973     54900
297222     50900
310134     58900
318164     55555
321332     65000
327855     53000
328541     59000
331623     53900
341005     699

In [18]:
autos.index

Int64Index([     1,      2,      3,      4,      5,      6,      8,      9,
                10,     11,
            ...
            371517, 371518, 371519, 371520, 371521, 371523, 371524, 371525,
            371526, 371527],
           dtype='int64', length=335320)

In [19]:
autos.index = range(335320)

As we removed data from the original data frame, it's cleaner to reset the index as we did in the cell above.

#### View of the various vehicle and fuel types

In [20]:
print(autos['vehicle_type'].unique(), autos['fuel_type'].unique())

['coupe' 'suv' 'kleinwagen' 'limousine' 'cabrio' 'bus' 'kombi' nan
 'andere'] ['diesel' 'benzin' nan 'lpg' 'andere' 'hybrid' 'cng' 'elektro']


#### We use a dictionary and the map function to convert vehicle and fuel types from German to English

In [21]:
mapping_vehicle_type = {
    'coupe': 'coupé',
    'kleinwagen': 'small car',
    'kombi': 'station wagon',
    'cabrio': 'convertible',
    'andere': 'other',
    'hybrid':'hybrid',
    'limousine':'limousine',
    'bus':'bus'
                        }

mapping_fuel_type = {
    'benzin': 'gasoline',
    'diesel': 'diesel',
    'lpg': 'liquefied petroleum gas',
    'andere': 'other',
    'cng': 'compressed natural gas',
    'elekto':'electric'
                    }

autos['vehicle_type'] = autos['vehicle_type'].map(mapping_vehicle_type)
autos['fuel_type'] = autos['fuel_type'].map(mapping_fuel_type)

#### Exploring the dates columns

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

In [22]:
autos[['date_crawled','ad_created','last_seen_online']].head()

,date_crawled,ad_created,last_seen_online
0,2016-03-24 10:58:45,2016-03-24 00:00:00,2016-04-07 01:46:50
1,2016-03-14 12:52:21,2016-03-14 00:00:00,2016-04-05 12:47:46
2,2016-03-17 16:54:04,2016-03-17 00:00:00,2016-03-17 17:40:17
3,2016-03-31 17:25:20,2016-03-31 00:00:00,2016-04-06 10:17:21
4,2016-04-04 17:36:23,2016-04-04 00:00:00,2016-04-06 19:17:07


You'll notice that the first 10 characters represent the day (e.g. 2016-03-12). To understand the date range, we can extract just the date values, use Series.value_counts() to generate a distribution, and then sort by the index.

To select the first 10 characters in each column, we can use Series.str[:10]

In [23]:
date_col = ['date_crawled', 'ad_created', 'last_seen_online']
for col in date_col:
    autos[col] = autos[col].str[:10]

In [24]:
autos['date_crawled'].value_counts(normalize=True).sort_index(ascending=True)

2016-03-05    0.025692
2016-03-06    0.014523
2016-03-07    0.035641
2016-03-08    0.033419
2016-03-09    0.034048
2016-03-10    0.032676
2016-03-11    0.032775
2016-03-12    0.036541
2016-03-13    0.015895
2016-03-14    0.036246
2016-03-15    0.033183
2016-03-16    0.030123
2016-03-17    0.031325
2016-03-18    0.013029
2016-03-19    0.035286
2016-03-20    0.036288
2016-03-21    0.035432
2016-03-22    0.032309
2016-03-23    0.032008
2016-03-24    0.029786
2016-03-25    0.032796
2016-03-26    0.032238
2016-03-27    0.030466
2016-03-28    0.035157
2016-03-29    0.033929
2016-03-30    0.033380
2016-03-31    0.031776
2016-04-01    0.034236
2016-04-02    0.035327
2016-04-03    0.039163
2016-04-04    0.037844
2016-04-05    0.012737
2016-04-06    0.003137
2016-04-07    0.001587
Name: date_crawled, dtype: float64

The cars were crawled each day between March 2016, 5th and March 2016, 7th. The repartition of the data collection is uniform, each day ~3% of the data were collected.

In [25]:
a = autos['ad_created'].value_counts(normalize=True).sort_index(ascending=False)
b = a[a > 0.02]
b

2016-04-04    0.037934
2016-04-03    0.039377
2016-04-02    0.035187
2016-04-01    0.034078
2016-03-31    0.031853
2016-03-30    0.033252
2016-03-29    0.033971
2016-03-28    0.035214
2016-03-27    0.030255
2016-03-26    0.032351
2016-03-25    0.032864
2016-03-24    0.029795
2016-03-23    0.031984
2016-03-22    0.031999
2016-03-21    0.035548
2016-03-20    0.036377
2016-03-19    0.034284
2016-03-17    0.030935
2016-03-16    0.030347
2016-03-15    0.033097
2016-03-14    0.035131
2016-03-12    0.036347
2016-03-11    0.032828
2016-03-10    0.032524
2016-03-09    0.034096
2016-03-08    0.033320
2016-03-07    0.034463
2016-03-05    0.022954
Name: ad_created, dtype: float64

The ads crawled were created between March 2015, 20th and April 2016, 7th. We notice that 93% of them were created between March 2016, 5th and April 2016, 7th. It's in line with the dates the website was crawled. We can infer that an ad has a lifetime of approximately one month. We could for example neglect the ads created before March 2016 depending on our analysis.

In [26]:
autos['last_seen_online'].value_counts(normalize=True).sort_index(ascending=False)

2016-04-07    0.132590
2016-04-06    0.222859
2016-04-05    0.128931
2016-04-04    0.025161
2016-04-03    0.025203
2016-04-02    0.024741
2016-04-01    0.023777
2016-03-31    0.023876
2016-03-30    0.023321
2016-03-29    0.022826
2016-03-28    0.021854
2016-03-27    0.016360
2016-03-26    0.015707
2016-03-25    0.018508
2016-03-24    0.018898
2016-03-23    0.017717
2016-03-22    0.020217
2016-03-21    0.019772
2016-03-20    0.019534
2016-03-19    0.016113
2016-03-18    0.006802
2016-03-17    0.028274
2016-03-16    0.016247
2016-03-15    0.015958
2016-03-14    0.011983
2016-03-13    0.008303
2016-03-12    0.023020
2016-03-11    0.012737
2016-03-10    0.011189
2016-03-09    0.009615
2016-03-08    0.007694
2016-03-07    0.004983
2016-03-06    0.004017
2016-03-05    0.001214
Name: last_seen_online, dtype: float64

~50% of the cars were last seen online during the three last days the website was crawled. We can infer that the crawler needs to put a value in the last_seen_online column even if the ad is still active. If we have to analyse the car sales, we will then exclude these three days from the analysis.

Let's check the consistency of the data and the reliability of the crawler by comparing the date the ad was created and the last time it was seen online:

In [27]:
(autos['ad_created'] <= autos['last_seen_online']).all()

True

All the ads were created before they were last seen online, then the crawling worked properly.

In [28]:
a = (autos['registration_year'] > 2016)
b = a[a == True]
b.count()

13183

Some of the data in the registration_year column are wrong, a registration superior to April 2016 is not possible based on the date of the latest crawling. Given  the important number of cars between 2017 and 2019, we could change these values to 2016 for example.  We can consider that a registration before 1945 is also an error. Even if some ads might be right, there is a high probabilty to find outliers.

In [29]:
cars_by_year = autos['registration_year'].value_counts().sort_index(ascending=False)
autos.loc[autos["registration_year"].between(2017,2018), "registration_year"] = 2016
print(cars_by_year.head(12))
print(cars_by_year.tail(12))

9999    10
9450     1
9000     4
8888     2
8500     1
8200     1
7800     1
7777     1
7500     1
7100     1
7000     3
6200     1
Name: registration_year, dtype: int64
1923     3
1911     1
1910    19
1800     2
1602     1
1600     2
1500     1
1400     1
1234     1
1039     1
1001     1
1000    15
Name: registration_year, dtype: int64


In [30]:
(~autos["registration_year"].between(1945,2016)).sum() / autos.shape[0]

0.0006232852200882739

The percentage of values that we are going to remove from the dataset is marginal (less than 0.1%)

In [31]:
# Many ways to select rows in a dataframe that fall within a value range for a column.
# Using `Series.between()` is one way.
autos = autos[autos["registration_year"].between(1945,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

2005    0.063430
2016    0.060377
2006    0.059503
2000    0.059443
1999    0.058458
2004    0.057763
2003    0.057757
2001    0.056268
2002    0.054907
2007    0.052063
Name: registration_year, dtype: float64

In [32]:
p_brands = autos['brand'].value_counts(normalize=True)
p_brands

volkswagen        0.213750
bmw               0.113497
mercedes_benz     0.101644
opel              0.100719
audi              0.092727
ford              0.064098
renault           0.044848
peugeot           0.029775
fiat              0.024270
seat              0.018716
skoda             0.016275
smart             0.015249
mazda             0.015144
citroen           0.013948
toyota            0.013572
nissan            0.013228
hyundai           0.010304
mini              0.009955
sonstige_autos    0.009155
volvo             0.009036
mitsubishi        0.007753
honda             0.007657
kia               0.007195
porsche           0.006284
suzuki            0.006258
alfa_romeo        0.006225
chevrolet         0.005282
chrysler          0.003861
dacia             0.002647
jeep              0.002307
land_rover        0.002259
subaru            0.001961
daihatsu          0.001829
jaguar            0.001784
saab              0.001462
daewoo            0.001244
lancia            0.001179
t

To carry out the analysis of the car price within each brand, we are going to consider only brands that represent at least 1% of the whole dataset, there are 16 brands. We need enough data to perform this kind of analysis to avoid mistaking due to an inconsistency of the data. A wrong price has more impact if the number of total prices is low.

In [33]:
brand_mean_price = {}
brands = p_brands[p_brands > 0.01].index
for b in brands:
    mean_price = int(autos.loc[autos.brand == b, "dollar_price"].mean())
    mean_mileage = int(autos.loc[autos.brand == b, "kilometer"].mean())
    mean_year = int(autos.loc[autos.brand == b, "registration_year"].mean())
    brand_mean_price[b] = [mean_price, mean_mileage, mean_year]
df = pd.DataFrame.from_dict(brand_mean_price, orient='index', columns=['mean_price', 'mean_mileage', 'mean_year'])
df.head()

,mean_price,mean_mileage,mean_year
volkswagen,5695,128152,2003
bmw,8574,133151,2003
mercedes_benz,8569,130925,2002
opel,3327,127640,2003
audi,9205,129651,2004


The mean prices are in line with our knowledge on the brand real prices. The three more expensive brands are respectively Audi, Bmw and Mercedes because of their high standards. More affordable brands like Renault, Fiat, Opel and Peugeot have a cheaper mean price. We could also add the mean year of registration per brand to improve the analysis. 

In [34]:
autos.to_csv(r'C:\Users\moham\Documents\Documents\DataQuest\Ebay Project\autos_new.csv', sep=',', index=False)